# Описательный анализ данных

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import path
import sys
sys.path.append(path.abspath('..'))

In [3]:
import os
from PIL import Image
import cv2
import random
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt

import json

import src.dataset_splitter as dspl
from src.dataset import PlateDataset
import src.augmentations as self_augs

ImportError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [ ]:
SAVE_PATH = 'single_COCO'
IMAGE_PATH = '../dataset/single_COCO'
COCO_ANNOTATION_PATH = '../dataset/COCO_plate_dataset.json'
DATA_PATH = '../dataset'

## Подбор и комбинация данных с kaggle

In [ ]:
image_data_dict = {
	'full_paths': [],
	'bboxes': [],
	'labels': [],
}

### Automatic-Number-Plate-Recognitions

https://www.kaggle.com/datasets/aslanahmedov/number-plate-detection

In [ ]:
# DATA_DIRS = list(os.walk('Automatic-Number-Plate-Recognition/images'))[0][-1]

all_files_path = 'Automatic-Number-Plate-Recognition/images'
all_files = list(os.walk(all_files_path))[0][-1]
len(all_files) / 2

In [ ]:
# Reading the data inside the xml
# file to a variable under the name 
# data
with open('Automatic-Number-Plate-Recognition/images/N2.xml', 'r') as f:
	data = f.read()

# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object 
Bs_data = BeautifulSoup(data, "xml")

# Finding all instances of tag 
# `unique`
b_unique = Bs_data.find_all('object')

print(b_unique)

# Using find() to extract attributes 
# of the first instance of the tag
b_name = Bs_data.find('filename')

print(b_name)

# Extracting the data stored in a
# specific attribute of the 
# `child` tag
value = b_name.get('test')

print(value)


In [ ]:
image_names = []
image_bboxes = []
for annot_file in tqdm(list(os.walk('Automatic-Number-Plate-Recognition/images'))[0][2]):
	if '.xml' in annot_file:
		# Reading the data inside the xml
		with open(os.path.join('Automatic-Number-Plate-Recognition/images/', annot_file), 'r') as f:
			data = f.read()
		
		# Passing the stored data 
		Bs_data = BeautifulSoup(data, "xml")
		
		# Finding image name 
		image_names.append(Bs_data.find('filename').text)
		
		# Finding image annotations
		objects = Bs_data.find_all('object')
		bboxes = []
		labels = []
		for obj in objects:
			bbox = [
				int(obj.find('bndbox').find('xmin').text),
				int(obj.find('bndbox').find('ymin').text),
				int(obj.find('bndbox').find('xmax').text),  # - int(obj.find('bndbox').find('xmin').text),
				int(obj.find('bndbox').find('ymax').text),  # - int(obj.find('bndbox').find('ymin').text)
			]
			bboxes.append(bbox)
			labels.append('plate')
		image_bboxes.append(bboxes)
		
		# Compile big dataset
		image_data_dict['full_paths'].append(
			os.path.join(all_files_path, Bs_data.find('filename').text)
		)
		image_data_dict['bboxes'].append(bboxes)
		image_data_dict['labels'].append(labels)
			

In [ ]:
print(image_data_dict['full_paths'][-1])
print(image_data_dict['bboxes'][-1])
print(image_data_dict['labels'][-1])

In [ ]:
# Plot and check data
random_i = random.randint(0, len(image_names) - 1)

test_image_path = os.path.join('Automatic-Number-Plate-Recognition/images', image_names[random_i])
test_image_bboxes = image_bboxes[random_i]

test_image = cv2.imread(str(test_image_path))
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image_bbox = test_image.copy()

for bbox in test_image_bboxes:
	cv2.rectangle(
		test_image_bbox, 
		(int(bbox[0]), int(bbox[1])),
		(int(bbox[2]), int(bbox[3])),
		(255,0,0),
		2
	)
		
plt.imshow(test_image_bbox);

###  Car-and-License-Plate-Detection

https://www.kaggle.com/datasets/riotulab/car-and-license-plate-detection

In [ ]:
all_files_path = 'Car-and-License-Plate-Detection/Car-and-License-plates-detection'
all_files = list(os.walk(all_files_path))[0][-1]
len(all_files) / 3

In [ ]:
image_names = []
image_bboxes = []
image_labels = []
for annot_file in tqdm(all_files):
	if '.xml' in annot_file:
		# Reading the data inside the xml
		with open(os.path.join(all_files_path, annot_file), 'r') as f:
			data = f.read()
		
		# Passing the stored data 
		Bs_data = BeautifulSoup(data, "xml")
		
		# Finding image name 
		image_names.append(Bs_data.find('filename').text)
		
		# Finding image annotations
		objects = Bs_data.find_all('object')
		bboxes = []
		labels = []
		for obj in objects:
			obj_name = obj.find('name').text
			if obj_name in ['LICENSE-PLATE', 'CAR']:
				labels.append(obj_name)
				bbox = [
					int(obj.find('bndbox').find('xmin').text),
					int(obj.find('bndbox').find('ymin').text),
					int(obj.find('bndbox').find('xmax').text),  # - int(obj.find('bndbox').find('xmin').text),
					int(obj.find('bndbox').find('ymax').text),  # - int(obj.find('bndbox').find('ymin').text)
				]
				bboxes.append(bbox)
		image_bboxes.append(bboxes)
		image_labels.append(labels)	
		
		# Compile big dataset
		image_data_dict['full_paths'].append(
			os.path.join(all_files_path, Bs_data.find('filename').text)
		)
		image_data_dict['bboxes'].append(bboxes)
		image_data_dict['labels'].append(labels)

In [ ]:
print(image_data_dict['full_paths'][-1])
print(image_data_dict['bboxes'][-1])
print(image_data_dict['labels'][-1])

In [ ]:
# Plot and check data
random_i = random.randint(0, len(image_names) - 1)

test_image_path = os.path.join(all_files_path, image_names[random_i])
test_image_bboxes = image_bboxes[random_i]
test_image_labels = image_labels[random_i]

test_image = cv2.imread(str(test_image_path))
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image_bbox = test_image.copy()

for bbox_i, bbox in enumerate(test_image_bboxes):
	cv2.rectangle(
		test_image_bbox, 
		(int(bbox[0]), int(bbox[1])),
		(int(bbox[2]), int(bbox[3])),
		(255,0,0),
		2
	)
	cv2.putText(
		test_image_bbox, 
		test_image_labels[bbox_i],
		(int(bbox[0]), int(bbox[1])), 
		cv2.FONT_HERSHEY_SIMPLEX, 
		fontScale=1, 
		color = (255, 0, 0),
		thickness=2
	)

plt.figure(figsize = (20,20))
plt.imshow(test_image_bbox);

###  Car-License-Plate-Detection

https://www.kaggle.com/datasets/amirhoseinahmadnejad/car-license-plate-detection-iran

In [ ]:
all_files_path = 'Car-License-Plate-Detection/train'
all_files_train = list(os.walk('Car-License-Plate-Detection/train'))[0][-1]
all_files_valid = list(os.walk('Car-License-Plate-Detection/valid'))[0][-1]
all_files_test = list(os.walk('Car-License-Plate-Detection/test'))[0][-1]
print(f"train: {len(all_files_train)}, validate: {len(all_files_valid)}, test: {len(all_files_test)}")

In [ ]:
image_names = []
image_bboxes = []
image_labels = []
for annot_file in [
	'Car-License-Plate-Detection/train/',
	'Car-License-Plate-Detection/valid/',
	'Car-License-Plate-Detection/test/'
]:
	# Reading the data inside the xml
	with open(os.path.join(annot_file, '_annotations.coco.json'), 'r') as json_file:
		json_data_dir = json.load(json_file)
		# Process image data
		for image_inf in tqdm(json_data_dir['images']):
			image_path = os.path.join(annot_file, image_inf['file_name'])
			image_names.append(image_path)
			image_id = image_inf['id']
			bboxes = []
			labels = []
			for annot in json_data_dir['annotations']:
				if annot['image_id'] == image_id:
					x_min, y_min, wight, height = annot['bbox']
					bboxes.append([int(x_min), int(y_min), int(x_min+wight), int(y_min+height)])
					labels.append('plate')
			image_bboxes.append(bboxes)
			image_labels.append(labels)
			
			# Compile big dataset
			image_data_dict['full_paths'].append(
				image_path
			)
			image_data_dict['bboxes'].append(bboxes)
			image_data_dict['labels'].append(labels)

print(image_names[-1])
print(image_bboxes[-1])
print(image_labels[-1])

In [ ]:
print(image_data_dict['full_paths'][-1])
print(image_data_dict['bboxes'][-1])
print(image_data_dict['labels'][-1])

In [ ]:
# Plot and check data
random_i = random.randint(0, len(image_names) - 1)

test_image_path = image_names[random_i]
test_image_bboxes = image_bboxes[random_i]
test_image_labels = image_labels[random_i]

test_image = cv2.imread(str(test_image_path))
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image_bbox = test_image.copy()

for bbox_i, bbox in enumerate(test_image_bboxes):
	cv2.rectangle(
		test_image_bbox, 
		(int(bbox[0]), int(bbox[1])),
		(int(bbox[2]), int(bbox[3])),
		(255,0,0),
		2
	)
	cv2.putText(
		test_image_bbox, 
		test_image_labels[bbox_i],
		(int(bbox[0]), int(bbox[1])), 
		cv2.FONT_HERSHEY_SIMPLEX, 
		fontScale=1, 
		color = (255, 0, 0),
		thickness=2
	)

plt.figure(figsize = (20,20))
plt.imshow(test_image_bbox);

###  Car plate object detection

https://www.kaggle.com/datasets/andrewteplov/car-plate-object-detetcion

In [ ]:
all_files_path = 'Car-plate-object-detection/train'
all_files_train = list(os.walk('Car-License-Plate-Detection/train'))[0][-1]
all_files_valid = '0'  # list(os.walk('Car-License-Plate-Detection/valid'))[0][-1]
all_files_test = '0'  # list(os.walk('Car-License-Plate-Detection/test'))[0][-1]
print(f"train: {len(all_files_train)}, validate: {len(all_files_valid)}, test: {len(all_files_test)}")

In [ ]:
df = pd.read_csv('Car-plate-object-detection/annotation_car_plate_train.csv')
print(df.shape)
df.head()

In [ ]:
all_data = {}

image_names = df['file'].to_numpy()
x_mins = df['xmin'].to_numpy()
y_mins = df['ymin'].to_numpy()
x_maxs = df['xmax'].to_numpy()
y_maxs = df['ymax'].to_numpy()

for image_i, image_name in enumerate(image_names):
	if image_name not in all_data:
		all_data[image_name] = {
			'bboxes': [[x_mins[image_i], y_mins[image_i], x_maxs[image_i], y_maxs[image_i]]],
			'labels': ['plate']
		}
	else:
		print('double')
		all_data[image_name]['bboxes'].append(
			[x_mins[image_i], y_mins[image_i], x_maxs[image_i], y_maxs[image_i]]
		)
		all_data[image_name]['labels'].append('plate')
		
# print(set([len(all_data[x]) for x in all_data]))

for image_name in all_data:
	# Compile big dataset
	image_data_dict['full_paths'].append(
		os.path.join(all_files_path, f'{image_name}.jpg')
	)
	image_data_dict['bboxes'].append(all_data[image_name]['bboxes'])
	image_data_dict['labels'].append(all_data[image_name]['labels'])
	
print(image_data_dict['full_paths'][-1])
print(image_data_dict['bboxes'][-1])
print(image_data_dict['labels'][-1])

In [ ]:
# Plot and check data
random_i = random.randint(0, image_names.shape[0] - 1)

test_image_path = os.path.join(all_files_path, f'{image_names[random_i]}.jpg')
x_min = x_mins[random_i]
y_min = y_mins[random_i]
x_max = x_maxs[random_i]
y_max = y_maxs[random_i]

test_image = cv2.imread(str(test_image_path))
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image_bbox = test_image.copy()

cv2.rectangle(
	test_image_bbox, 
	(int(x_min), int(y_min)),
	(int(x_max), int(y_max)),
	(255,0,0),
	20
)
# cv2.putText(
# 	test_image_bbox, 
# 	test_image_labels[bbox_i],
# 	(int(bbox[0]), int(bbox[1])), 
# 	cv2.FONT_HERSHEY_SIMPLEX, 
# 	fontScale=1, 
# 	color = (255, 0, 0),
# 	thickness=2
# )

plt.figure(figsize = (20,20))
plt.imshow(test_image_bbox);

# Сборка единого датасета в COCO формате

In [ ]:
print(image_data_dict['full_paths'][-1])
print(image_data_dict['bboxes'][-1])
print(image_data_dict['labels'][-1])

print(len(image_data_dict['full_paths']))
print(len(image_data_dict['bboxes']))
print(len(image_data_dict['labels']))

total_labels = []
for labels in image_data_dict['labels']:
	for l in labels:
		if l not in total_labels:
			total_labels.append(l)
			
print(total_labels)

In [ ]:
COCO_plate_dataset = {
    'licenses': [
        {
            'name': 'LogicYield MIT', 'id': 0, 'url': ''
        }
    ], 
    'info': {
        'contributor': "LLC LogicYield", 
        'date_created': "28-04-2024", 
        'description': "Dataset with selected plate numbers", 
        'url': "", 
        'version': "1.0", 
        'year': "2024"
    }, 
    'categories': [
        {'id': 1, 'name': 'plate', 'supercategory': ''},
		{'id': 2, 'name': 'car', 'supercategory': ''},
    ], 
    'images': [], 
    'annotations': []
}

In [ ]:
for image_i, image_path in enumerate(tqdm(image_data_dict['full_paths'])):
	image = cv2.imread(str(image_path))
	# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	cv2.imwrite(
		os.path.join(SAVE_PATH, f"{image_i}.jpg"), 
		image
	)  # Save image
	size = image.shape
	image_coco_data_i = {
		'id': image_i, 
		'width': size[1], 
		'height': size[0],
		'file_name': f"{image_i}.jpg",
		'license': 0, 
		'flickr_url': '', 
		'coco_url': '', 
		'date_captured': '28-04-2024'
	}
	COCO_plate_dataset['images'].append(image_coco_data_i)
	for bbox_i, bbox in enumerate(image_data_dict['bboxes'][image_i]):
		label = image_data_dict['labels'][image_i][bbox_i]
		label_id = 1
		if label in ['CAR']:
			label_id = 2
		x_min, y_min, x_max, y_max = bbox
		annotation_coco_data_i = {
			'id': bbox_i,  # self id in image
			'image_id': image_i,  # according to images
			'category_id': label_id,  # according to categorise
			'segmentation': [], 
			'area': round(int(x_max-x_min) * int(y_max-y_min)), 
			'bbox': [int(x_min), int(y_min), int(x_max-x_min), int(y_max-y_min)], 
			'iscrowd': 0
		}
		COCO_plate_dataset['annotations'].append(annotation_coco_data_i)

In [ ]:
# Save COCO
json_object = json.dumps(COCO_plate_dataset, indent=4)

with open('COCO_plate_dataset.json', 'w') as fp:
    fp.write(json_object)

# Тестирование модулей для модели

## Тестирование СОСО датасета

In [ ]:
with open(COCO_ANNOTATION_PATH, 'r') as json_data:
    plate_COCO = json.load(json_data)

print(plate_COCO.keys())

In [ ]:
# Test COCO data
# Select random image in COCO
all_imgs_data = len(plate_COCO['images'])
test_i = random.randint(0, all_imgs_data-1)

select_image_data = plate_COCO['images'][test_i]
image_id = select_image_data['id']
image_path = os.path.join(IMAGE_PATH, select_image_data['file_name'])

# Load image
test_image = cv2.imread(image_path)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

valid_annot = []
for i in plate_COCO['annotations']:
    if i['image_id'] == image_id:
        valid_annot.append(i)

# print(
#     (f"len test segments: {len(valid_annot[0]['segmentation'])}," 
#     f"len points: {len(points_xy)}, len bboxes: {len(COCO_dataset_UCCV['annotations'][test_i]['bbox'])}")
# )

# Plot COCO data on image
for annotation in valid_annot:
    bbox = annotation["bbox"]
    
    cv2.rectangle(
        test_image, 
        (int(bbox[0]), int(bbox[1])),
        (int(bbox[0]+bbox[2]), int(bbox[1]+bbox[3])),
        (255,0,0),
        4
    )

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(test_image, cmap='gray', vmin=0, vmax=255, interpolation='nearest')

## Тестирование разбиения данных

In [ ]:
(
	train_data_dict,
	validate_data_dict,
	test_data_dict,
	classes
) = dspl.preprocess_and_split(
	data_path=DATA_PATH,
	valid_part = 0.2,
	test_part = 0.1
)

In [ ]:
train_data_dict['Labels'][186]

## Тестирование датасета и аугментаций

In [ ]:
pure_dataset = PlateDataset(
	image_dir = IMAGE_PATH, 
	image_paths = train_data_dict['Full_paths'],
	image_bboxes = train_data_dict['Bboxes'],
	image_labels = train_data_dict['Labels'],
	classes=['plate'],
	transforms = None,
	preprocessing = None
)

In [ ]:
random_i = random.randint(0, len(pure_dataset) - 1)
print(
	f"Selected index {random_i} from {len(pure_dataset)}"
)

image, mask = pure_dataset[random_i]
print(f"Image shape: {image.shape}, mask shape: {mask.shape}")


plt.rcParams['figure.figsize'] = [20, 10]
f, axarr = plt.subplots(1, 2) 

axarr[0].imshow(Image.fromarray(image), cmap='gray', vmin=0, vmax=255)
axarr[1].imshow(mask[:, :, 0], cmap='gray', vmin=0, vmax=1)
# 	
# 	
# Image.fromarray(image)
# Image.fromarray(mask)
# # plt.imshow()

In [ ]:
transform_train = self_augs.get_transforms(
	width=640,
	height=640,
	preprocessing=True,
	augmentations=True,
	postprocessing=True
)

transform_validate = self_augs.get_transforms(
	width=640,
	height=640,
	preprocessing=True,
	augmentations=False,
	postprocessing=True
)

train_dataset = PlateDataset(
        image_dir = IMAGE_PATH, 
		image_paths = train_data_dict['Full_paths'],
		image_bboxes = train_data_dict['Bboxes'],
		image_labels = train_data_dict['Labels'],
		classes=['plate'],
        transforms = transform_train
    )
validate_dataset = PlateDataset(
        image_dir = IMAGE_PATH, 
		image_paths = validate_data_dict['Full_paths'],
		image_bboxes = validate_data_dict['Bboxes'],
		image_labels = validate_data_dict['Labels'],
		classes=['plate'],
        transforms = transform_validate
    )

In [ ]:
for _test_dataset in [train_dataset, validate_dataset]:
	random_i = random.randint(0, len(_test_dataset) - 1)
	print(
		f"Selected index {random_i} from {len(_test_dataset)}"
	)
	
	test_image, test_mask = _test_dataset[random_i]
	print(f"Image shape: {test_image.shape}, mask shape: {test_mask.shape}")
	
	test_image = self_augs.denormalize(self_augs.tensor_to_cv_image(test_image))
	# test_mask = np.transpose(test_mask, (1,2,0))
	
	plt.rcParams['figure.figsize'] = [20, 10]
	f, axarr = plt.subplots(1, 2) 
	
	axarr[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
	axarr[1].imshow(test_mask[:, :, 0], cmap='gray', vmin=0, vmax=1)
	# 	
	# 	
	# Image.fromarray(image)
	# Image.fromarray(mask)
	# # plt.imshow()

In [ ]:
train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=32,
            num_workers=10,
            shuffle=True,
            pin_memory=True,
            drop_last=False,
        )
valid_loader = DataLoader(
            dataset=validate_dataset,
            batch_size=1,
            num_workers=10,
            shuffle=False,
            pin_memory=True,
            drop_last=False,
        )

In [ ]:
# Test dataloader
for _test_dataloader in [train_loader, valid_loader]:
    print('process')
    test_image, test_mask = next(iter(_test_dataloader))
    test_image = self_augs.denormalize(self_augs.tensor_to_cv_image(test_image[0]))
    
    plt.rcParams['figure.figsize'] = [20, 10]
    f, axarr = plt.subplots(1, 2) 
    
    axarr[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
    axarr[1].imshow(test_mask[0][:, :, 0], cmap='gray', vmin=0, vmax=1)
    print()